In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Load the modified CSV file
file_path = 'NewsResult_20240812-20240812_modified_titles.csv'
df_new = pd.read_csv(file_path)

# Load the saved model and tokenizer
model_path = "klue-bert-classificationTitles-best-model"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

# Tokenize and encode the titles from the new CSV
texts = df_new['title'].tolist()  # Extract the titles
encoded_data = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)

# Convert predictions back to labels (0 or 1)
predictions = preds.cpu().numpy()
df_new['predictions'] = predictions  # Add predictions to the DataFrame

# # Save the DataFrame with predictions
# output_file = 'predictions_with_titles.csv'
# df_new.to_csv(output_file, index=False)

# print(f"Predictions saved to {output_file}")


In [ ]:
# Save the DataFrame with predictions
output_file = 'predictions_with_titles.csv'
df_new.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")


In [8]:
df_new

,title,predictions
0,"'대변화 예고'하는 민주당 전북도 당정협의 이원택 ""당 주도에 균형점 맞출 것""",0
1,"尹 대통령, 이명박 전 대통령과 부부 동반 첫 만찬",0
2,"윤 통화내역 확보한 공수처, ‘채상병 사건 외압’ 단서 잡을까",0
3,"윤 대통령, 신임 국가인권위원장 후보에 안창호 전 헌재재판관 지명",0
4,윤 대통령 “전기차 화재 코로나 확산에 신속하게 대처하라” 지시,0
...,...,...
2136,여야 ‘한지붕 두가족’ 계파 꿈틀 친윤 친명 ‘촉각’,0
2137,"민주당 대전시당위원장에 박정현 의원 ""새 역사 써나갈 것""",0
2138,"경기도, 도의회와 함께 기회소득 정착 모색",0
2139,"구자근 의원, 지방경제 기업 살리기 시리즈 2탄 패키지 법안 발의",0
